# Pandas Dataframeの扱い方

In [1]:
import pandas as pd
import numpy as np


## dataframe作成

In [12]:
print(pd.__version__)
# 2.1.4

df = pd.DataFrame({'age': [24, 42], 'state': ['NY', 'CA'], 'point': [64, 92]},
                  )

# 以下のようにindexを指定してdataframeを作成することも可能。だが、他人が見たときに直観に反するため、明確に使う理由がある場合を除き、使わないほうがよさそう。
# そもそも大量データでソートキーとしてindexを使いたいとすると、pythonの前にdbを置くべきか
#df = pd.DataFrame({'age': [24, 42], 'state': ['NY', 'CA'], 'point': [64, 92]},
#                  index=['Alice', 'Bob'])

df

2.1.1


,age,state,point
0,24,NY,64
1,42,CA,92


### forループ
dataframeをそのままforループに回すとカラムが返ってくる


In [13]:
for column_name in df:
    print(column_name)


age
state
point


### 1列ずつ取り出す: items()
カラム毎にカラム名とデータのリストが引き渡される

‐ https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.items.html

いつ使うんでしょうか？

In [18]:
for column_name, item in df.items():
    print(column_name)
    print(type(item))
    print(item)
    print('======')

age
<class 'pandas.core.series.Series'>
0    24
1    42
Name: age, dtype: int64
state
<class 'pandas.core.series.Series'>
0    NY
1    CA
Name: state, dtype: object
point
<class 'pandas.core.series.Series'>
0    64
1    92
Name: point, dtype: int64


# 1行ずつ取り出す: iterrows(), itertuples()
DataFrameから1行ずつ取り出すメソッドにはiterrows()とitertuples()がある。itertuples()のほうが高速。

特定の列の値のみが必要なのであれば、次に説明するように列を指定して個別にforループで回すほうがさらに速い。処理速度についての実験結果は最後にす 

## iterrows()メソッド
行番号と行データを別々に渡してくれるので便利

In [21]:
for index, row in df.iterrows():
    print("index:%d" % index)
    print(type(row))
    print(row['age'], row['state'], row['point'])
    print('======')


index:0
<class 'pandas.core.series.Series'>
24 NY 64
index:1
<class 'pandas.core.series.Series'>
42 CA 92


## itertuples()メソッド
itertuples()メソッドを使うと、行名とその行のデータ（namedtuple）のタプル(index, namedtuple)を1行ずつ取得できる。

pandas.DataFrame.itertuples — pandas 2.1.4 documentation
デフォルトではPandasという名前のnamedtuple（名前付きタプル）を返す。最初の要素が行名となる。namedtupleなので、[]のほか.でも各要素の値にアクセスできる。

In [22]:
for row in df.itertuples():
    print(type(row))
    print(row)
    print(row[0], row[1], row[2], row[3])
    print(row.Index, row.age, row.state, row.point)
    print('======')


<class 'pandas.core.frame.Pandas'>
Pandas(Index=0, age=24, state='NY', point=64)
0 24 NY 64
0 24 NY 64
<class 'pandas.core.frame.Pandas'>
Pandas(Index=1, age=42, state='CA', point=92)
1 42 CA 92
1 42 CA 92


row[0]=row.indexがポイント。先頭のデータが配列の先頭ではないところは注意。

また、引数`index`を`False`にすると`index`は要素に含まれない。また、引数`name`で`namedtuple`の名前を指定できる。



In [24]:
for row in df.itertuples(index=False, name='Person'):
    print(type(row))
    print(row)
    print(row[0], row[1], row[2])
    print(row.age, row.state, row.point)
    print('======')


<class 'pandas.core.frame.Person'>
Person(age=24, state='NY', point=64)
24 NY 64
24 NY 64
<class 'pandas.core.frame.Person'>
Person(age=42, state='CA', point=92)
42 CA 92
42 CA 92


データからIndexが消えている。行番号が必要ない場合はこれを使ってもよいかもしれない。

また、大量データを対象とする場合は、データ量削減のためにindexを削除してループする、とか言いそうだが、大量データの場合はそもそも対象カラムだけ抜き出して処理すべき。
だとすると、indexが邪魔な時に指定する以外、使いどころがない気がしている。

また、引数`name`を`None`にするとノーマルのタプルを返す。`index=False`よりもこっちを使うべきなきがする。

In [26]:
for row in df.itertuples(name=None):
    print(type(row))
    print(row)
    print(row[0], row[1], row[2], row[3])
    print('======')




<class 'tuple'>
(0, 24, 'NY', 64)
0 24 NY 64
<class 'tuple'>
(1, 42, 'CA', 92)
1 42 CA 92


これだと、行番号もデータもとれるタプルが取れる。ここで、index=Falseを指定すると、普通のforループみたいになる

In [29]:
for row in df.itertuples(name=None,index=False):
    print(type(row))
    print(row)
    print(row[0], row[1], row[2])
    print('======')


<class 'tuple'>
(24, 'NY', 64)
24 NY 64
<class 'tuple'>
(42, 'CA', 92)
42 CA 92


ちなみに、普通のforループと、forループで要素番号を取得する方法はこちら

In [52]:
seasons = ['Spring', 'Summer', 'Fall', 'Winter']

for index, item in enumerate(seasons):
    print(f"{index}, {item}")


0, Spring
1, Summer
2, Fall
3, Winter


# Dataframeにカラムを追加する

### dataframeの初期化

In [67]:
df = pd.DataFrame({'age': [24, 42], 'state': ['NY', 'CA'], 'point': [64, 92]},
                  )
df

,age,state,point
0,24,NY,64
1,42,CA,92


### 元のdataframeを更新してしまってよい場合

In [70]:
df = pd.DataFrame({'age': [24, 42], 'state': ['NY', 'CA'], 'point': [64, 92]},)
print("df")
print(df)

df["rank"] = 1

print("df（カラムが追加されている）")
print(df)


元のdf
   age state  point  rank
0   24    NY     64     1
1   42    CA     92     1
元のdfが更新されている


### 元のdataframeを更新したくない場合は、assignメソッドを使う

In [72]:
df = pd.DataFrame({'age': [24, 42], 'state': ['NY', 'CA'], 'point': [64, 92]},)
print("元のdf")
print(df)

df2 = df.assign(rank = 1)
print("assignメソッド呼出し後も、元のdataframeは変化はない")
print(df)
print("assignメソッドで新しいオブジェクトが返却されている")
print(df2)

元のdf
   age state  point
0   24    NY     64
1   42    CA     92
assignメソッド呼出し後も、元のdataframeは変化はない
   age state  point
0   24    NY     64
1   42    CA     92
assignメソッドで新しいオブジェクトが返却されている
   age state  point  rank
0   24    NY     64     1
1   42    CA     92     1


### カラム追加の際に、三項演算子で既存カラムの値をもとに新しい値を作り、カラムを追加する

In [78]:
df = pd.DataFrame({'age': [24, 42, 38], 'state': ['NY', 'CA', 'TX'], 'point': [64, 92, 48]},)
print("元のdf")
print(df)

# これだとエラーになる
df2 = df.assign(rank = "A" if df.point > 80 else "B")

print(df2)


元のdf
   age state  point
0   24    NY     64
1   42    CA     92
2   38    TX     48


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [82]:
df = pd.DataFrame({'age': [24, 42, 38], 'state': ['NY', 'CA', 'TX'], 'point': [64, 92, 48]},)
print("元のdf")
print(df)

df2 = df.assign(rank = df.point.apply(lambda x : "A" if x > 80 else "B" if x > 50 else "C") )

print(df2)

元のdf
   age state  point
0   24    NY     64
1   42    CA     92
2   38    TX     48
   age state  point rank
0   24    NY     64    B
1   42    CA     92    A
2   38    TX     48    C
